In [ ]:
#@title Initialization { vertical-output: true, display-mode: "form" }
!pip install pyunpack
!pip install patool
!pip install -Iv Pillow==9.3.0
!pip install colour
import subprocess, os
os.system("rm -r *")
os.system("git clone https://github.com/dphdmn/15puzzleSolverForGC")
os.system("git clone https://github.com/dphdmn/LMscramblesProject")
from LMscramblesProject import scramblesExtract, scramblesParser, solutionsInsert, statsAnalyser
from google.colab import files
import shutil
from pyunpack import Archive
os.chdir("/content")
os.mkdir('replays')
os.mkdir('inputFile')
os.mkdir('outputFiles')
os.chdir("15puzzleSolverForGC")
os.system("chmod +x solver3")
os.system("chmod +x solver4")
os.system("chmod +x run.sh")
os.chdir("/content")
outputFile="outputFiles/output.txt"
inputFile="inputFile/input.txt"
statsFile="outputFiles/stats.txt"
imageFile="outputFiles/image.png"
replaysFolder="replays/"
scramblesPath="15puzzleSolverForGC/scrambles"
solutionsPath="15puzzleSolverForGC/solutions"

In [ ]:
#@title Choose ZIP archive of your replays (please don't use subfolders, and only add replay files in archive)
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = filename
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)
  Archive(filename).extractall(replaysFolder)
  os.system("rm " + filename)

In [ ]:
#@title Choose your solve file (standard LM export output as txt file, without the header line, only solves list, 1 file)
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = inputFile
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

In [8]:
#@title Parse scrambles, solve, and generate output/stats file, before running enter GMT timezone in hours (You can find stats with scrambles added in outputFiles folder)
timezone = 8 #@param {type:"number"}
scramblesParser.parse(replaysFolder, inputFile, outputFile, -timezone)
scramblesExtract.extractScrambles(outputFile, scramblesPath)
os.chdir("15puzzleSolverForGC")
subprocess.call("./run.sh", shell=True)
solutions=[]
with open("solutions", "r") as f:
  d = f.readlines()
  for i in d:
    if i != "done\n" and i != "\n":
      solutions.append(i)
with open("solutions", "w") as f:
  for s in solutions:
    f.write(s)
os.chdir("/content")
solutionsInsert.insertSolutions(solutionsPath, outputFile, statsFile)

In [ ]:
#@title Analyse stats and get the image! (choose resolution, optimal = 2000)
resolution = 2000 #@param {type:"integer"}
fontpath = "LMscramblesProject/calibri_font.ttf"
statsAnalyser.analyseAndRender(statsFile, resolution, imageFile, fontpath)
from IPython.display import Image as dispImg
dispImg(imageFile)